In [3]:
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, Flatten, Dense
import cv2
from keras.models import load_model
from matplotlib import pyplot as plt
from keras.preprocessing.image import img_to_array

# Example data
input_data = np.random.rand(1, 48,48,1)  # Replace with your actual input data shape

# Create a model with convolutional layers and Flatten
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))
# Add more convolutional layers as needed
model.add(Flatten())




# Get the flattened output
image_path = 'D:\\Downloads\\Emotion Detection\\Emotion_Detection_CNN-main\\images\\cv6.png'
image = np.expand_dims(cv2.imread(image_path, 0), axis=2)
# image = cv2.imread(image_path)
image = cv2.resize(image, (48, 48),interpolation = cv2.INTER_AREA)
# plt.imshow(image)
image = image.astype('float')/255.0
image = img_to_array(image)
# plt.imshow(image)
image = np.expand_dims(image,axis=0)
flattened_output = model.predict(image)

# Print the shape of the flattened output
print("Shape after flattening:", flattened_output.shape)

# Optionally, print the actual flattened values
print("Flattened Output:", flattened_output[0][40000:40005])
# Example flattened vector
flattened_vector = flattened_output  # Replace 100 with the size of your flattened vector

# Create a simple model with a fully connected layer
model1 = Sequential()
model1.add(Dense(units=64, activation='relu', input_shape=(None,1,67712)))  # Change 100 to the size of your flattened vector
# Add more layers if needed
model1.add(Dense(units=5, activation='softmax'))  # Output layer with 10 units (example)

# Compile the model (you can customize the optimizer, loss, and metrics)
model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Predict using the model
raw_scores = model1.predict(flattened_vector)

# Print the raw scores
print("Raw Scores:", raw_scores)




1/1 [==============================] - 0s 245ms/step
Shape after flattening: (1, 67712)
Flattened Output: [0.01292237 0.02385195 0.01220972 0.00762045 0.0059512 ]

1/1 [==============================] - 0s 62ms/step
Raw Scores: [[0.21123864 0.192191   0.19746213 0.20174602 0.19736221]]


In [9]:
from keras import backend as K
moodel = load_model('D:\\Downloads\\Emotion Detection\\Emotion_Detection_CNN-main\\model_dcnn.keras')
sample_img = 'D:\\Downloads\\Emotion Detection\\Emotion_Detection_CNN-main\\images\\happy1.png'
image = np.expand_dims(cv2.imread(sample_img, 0), axis=2)
# image = cv2.imread(image_path)
image = cv2.resize(image, (48, 48),interpolation = cv2.INTER_AREA)
# plt.imshow(image)
image = image.astype('float')/255.0
image = img_to_array(image)
# plt.imshow(image)
image = np.expand_dims(image,axis=0)
preds = moodel.predict(image)
pred_vector_output = moodel.output[:, np.argmax(preds[0])]
heatmaps = []
INTERESTED_CONV_LAYERS = ["conv2d_1", "conv2d_2", "conv2d_3", "conv2d_4", "conv2d_5", "conv2d_6"]
layer_list = [(layer.name, layer) for layer in moodel.layers if "conv" in layer.name]
for layer in layer_list:
    if layer[0] in INTERESTED_CONV_LAYERS:
        some_conv_layer = moodel.get_layer(layer[0])
        grads = K.gradients(pred_vector_output, some_conv_layer.output)[0]
        pooled_grads = K.mean(grads, axis=(0, 1, 2))
        iterate = K.function([moodel.input], [pooled_grads, some_conv_layer.output[0]])
        pooled_grads_value, conv_layer_output_value = iterate([sample_img])

        for i in range(moodel.get_layer(layer[0]).output_shape[-1]):
            conv_layer_output_value[:, :, i] *= pooled_grads_value[i]

        heatmaps.append(np.mean(conv_layer_output_value, axis=-1))
fig = plt.figure(figsize=(14, 3))



1/1 [==============================] - 0s 170ms/step


RuntimeError: tf.gradients is not supported when eager execution is enabled. Use tf.GradientTape instead.

In [ ]:
for i, (name,hm) in enumerate(zip(INTERESTED_CONV_LAYERS, heatmaps)):
    ax = plt.subplot(1, 6, i+1)
    img_heatmap = np.maximum(hm, 0)
    img_heatmap /= np.max(img_heatmap)
    ax.imshow(img_heatmap, cmap="gray")
    ax.set_xticks([])
    ax.set_yticks([])
    plt.title(name)
    plt.tight_layout()